In [1]:
import pandas as pd

import dill

def load_data(f):
    with open(f,'rb') as infile:
        return dill.load(infile)

day_0,day_baseline,day_7,day_90,day_365,day_1,base_df = [load_data(f) for f in ['day_0_data.p',
                                                                        'day_baseline_data.p',
                                                                        'day_7_data.p',
                                                                        'day_90_data.p',
                                                                        'day_365_data.p',
                                                                        'day_1_data.p', 
                                                                        'base_df_data.p']]
scans = pd.read_pickle('scans.p')

In [2]:
for day in [day_0,day_baseline,day_7,day_90,day_365,day_1]:
    print([t for t in day.keys()])

['bloods', 'laboratory_tests']
['physical_examination', 'NIHSS_multiple', 'recent_medical_history', 'past_medical_history', 'CCMI', 'further_stroke_event_part', 'NIHSS', 'pre-stroke_mRS', 'bloods', 'laboratory_tests', 'radiological_scans', 'concomitant_medications']
['bloods', 'NIHSS', 'history_of_depression', 'MoCA', 'MADRS', 'physical_risk_factors', 'diet_q', 'RAPA', 'laboratory_tests', 'infarct_type', 'concomitant_medications', 'concomitant_medications_plus']
['bloods', 'mRS', 'NIHSS', 'BI', 'SIS', 'physical_examination', 'physical_risk_factors', 'RAPA', 'WSAS', 'diet_q', 'laboratory_tests', 'further_stroke_event_part', 'history_of_depression', 'MoCA', 'MADRS', 'concomitant_medications', 'MMSE', 'TUGT', 'digit_span', 'RAVENs', 'STROOP', 'shape_cancellation', 'TDT', 'ARAT', 'TUGT_average', 'ACS', 'MRI', 'lesion_location_confirmed']
['bloods', 'mRS', 'NIHSS', 'BI', 'SIS', 'physical_examination', 'physical_risk_factors', 'RAPA', 'WSAS', 'diet_q', 'laboratory_tests', 'further_stroke_eve

In [3]:
first = set(day_90['MRI'].conducted[day_90['MRI'].conducted > 0].index.values)
second = set(day_365['MRI'].conducted[day_365['MRI'].conducted > 0].index.values)
first_only = first.difference(second)
second_only = second.difference(first)
both = first.intersection(second)
either = first.union(second)

scan_done = pd.DataFrame({'baseline': pd.Series({x:0 for x in range(119,219)}),
                          'day_90_scan' : pd.Series({f:1 for f in first}),
                          'day_365_scan' : pd.Series({s:1 for s in second})})

In [4]:
sheet_0 = pd.read_excel('masks/lists.xlsx',sheetname=0)
sheet_1 = pd.read_excel('masks/lists.xlsx',sheetname=1)
sheet_2 = pd.read_excel('masks/lists.xlsx',sheetname=2)

In [5]:
sheet_0.drop(labels = 'Unnamed: 2',axis = 'columns',inplace=True)

In [6]:
sheet_1.drop(labels='Unnamed: 4',axis='columns',inplace=True)

In [7]:
sheet_2.drop(labels='Unnamed: 2',axis='columns',inplace=True)

In [8]:
def path_split(x):
    return x.rsplit('/',1)[1]
    

In [9]:
sheet_1['Out list'] = sheet_1['Out list'].apply(path_split)

In [10]:
scan_map = pd.DataFrame(sheet_0['Identifiable ID'])

In [11]:
scan_map['code'] = sheet_1['Out list']

In [12]:
scan_map['code'].shape

(125,)

In [13]:
files = []

with open('masks/files.txt') as infile:
    for line in infile:
        files.append(line.strip())

In [14]:
files.pop(-1)

'files.txt'

In [15]:
len(files)

125

In [16]:
len(first) + len(second)

124

In [17]:
for f in files:
    if f not in scan_map['code'].values:
        print(f )

D_PP_P01-23-BM
D_PP_P02-09-MG


In [17]:
codes_corr_maps = []
for x in [x for x in scans['path'].values]:
    for y in x[:2]:
        codes_corr_maps.append(y.split('/')[-1].rsplit('_global_correlation_aal.csv')[0])

In [18]:
len(codes_corr_maps)

122

In [44]:
for c in scan_map['code']:
    
    if c not in codes_corr_maps:
        if c.endswith('_12months'):
            start = c.split('_12months')[0]
            end = '_12months'
            alt_name = start[::-1].replace('_','-',1)[::-1] + end
        else:
            alt_name = c[::-1].replace('_','-',1)[::-1]
        if alt_name in codes_corr_maps:
            print('{} in corr_maps as {}'.format(c,alt_name))
        else:
            print('***{}/{}  not in corr_maps'.format(c,alt_name))
        print()

        

***D_???/D-???  not in corr_maps

***D_???/D-???  not in corr_maps

D_PP_P15-12_SR in corr_maps as D_PP_P15-12-SR

D_PP_P15-12_SR_12months in corr_maps as D_PP_P15-12-SR_12months

***D_PP_P15-36-JR/D_PP-P15-36-JR  not in corr_maps

***D_PP_P15-36-JR_12months/D_PP-P15-36-JR_12months  not in corr_maps

D_PP_P15-14_PS in corr_maps as D_PP_P15-14-PS

D_PP_P15-14_PS_12months in corr_maps as D_PP_P15-14-PS_12months

D_PP_P01-19_JS in corr_maps as D_PP_P01-19-JS

D_PP_P01-19_JS_12months in corr_maps as D_PP_P01-19-JS_12months

D_PP_P15-04_KT in corr_maps as D_PP_P15-04-KT

D_PP_P15-04_KT_12months in corr_maps as D_PP_P15-04-KT_12months



In [22]:
for c in codes_corr_maps:
    if c not in scan_map['code'].values:
        print(c)

D_PP_P15-04-KT_12months
D_PP_P15-04-KT
D_PP_P01-19-JS_12months
D_PP_P01-19-JS
D_PP_P15-12-SR_12months
D_PP_P15-12-SR
D_PP_P15-14-PS_12months
D_PP_P15-14-PS
D_PP_P01-23-BM


In [104]:
for c in scan_map['code'].to:
        print(c)
for c in codes_corr_maps:
    if c not in scan_map['code'].values:
        print(c)

D_PP_P01-21-CA
D_PP_P01-21-CA_12months
D_PP_P02-20-RA
D_PP_P02-20-RA_12months
D_PP_P15-03-MB
D_PP_P15-03-MB_12months
D_PP_P01-38-AB
D_PP_P01-38-AB_12months
D_PP_P01-17-IB
D_PP_P01-17-IB_12months
D_PP_P01-24-MB
D_PP_P01-30-WC
D_PP_P01-42-CC
D_PP_P01-42-CC_12months
D_PP_P01-14-GC_12months
D_PP_P15-05-BC
D_PP_P15-05-BC_12months
D_PP_P02-24-AD
D_PP_P02-24-AD_12months
D_PP_P01-39-WD
D_PP_P01-39-WD_12months
D_PP_P15-31-RE
D_PP_P15-31-RE_12months
D_PP_P02-10-DE
D_PP_P02-10-DE_12months
D_PP_P12-04-KE
D_PP_P12-04-KE_12months
D_PP_P02-06-KF
D_PP_P02-06-KF_12months
D_PP_P15-13-RF
D_PP_P15-13-RF_12months
D_PP_P15-29-DG_12months
D_???
D_PP_P01-25-MH
D_PP_P01-25-MH_12months
D_PP_P15-30-AH
D_PP_P15-30-AH_12months
D_PP_P02-02-SH
D_PP_P02-02-SH_12months
D_PP_P02-03-MH
D_PP_P02-03-MH_12months
D_PP_P01-41-JJ
D_PP_P01-41-JJ_12months
D_PP_P15-27-AJ
D_PP_P15-27-AJ_12months
D_PP_P02-15-MK
D_PP_P15-02-DK
D_PP_P15-02-DK_12months
D_PP_P01-43-GK_12months
D_PP_P01-43-GK
D_PP_P15-28-HK
D_PP_P15-28-HK_12months
D_PP

In [23]:
day_labels = 'day_0,day_baseline,day_7,day_90,day_365,day_1'.split(',')

In [61]:
with pd.HDFStore('test.h5') as store:
    for label,day in zip(day_labels,[day_0,day_baseline,day_7,day_90,day_365,day_1]):
        for k in day.keys():
            print('{}/{}'.format(label,k))
            print(day[k].head())
            store.put('{}/{}'.format(label,k),day[k])

day_0/bloods
   Chol  Trig  HDLC  LDLC  sFol  B12  CRP
0   NaN   NaN   NaN   NaN   NaN  NaN  NaN
1   NaN   NaN   NaN   NaN   NaN  NaN  NaN
2   NaN   NaN   NaN   NaN   NaN  NaN  NaN
3   NaN   NaN   NaN   NaN   NaN  NaN  NaN
4   NaN   NaN   NaN   NaN   NaN  NaN  NaN
day_0/laboratory_tests
  date_time  hemoglobin  white_cell_count  platelet_count  mcv  neutrophil  \
0       NaT         NaN               NaN             NaN  NaN         NaN   
1       NaT         NaN               NaN             NaN  NaN         NaN   
2       NaT         NaN               NaN             NaN  NaN         NaN   
3       NaT         NaN               NaN             NaN  NaN         NaN   
4       NaT         NaN               NaN             NaN  NaN         NaN   

   sodium  potassium  urea  creatinine  inr  egfr  aptt  pt  glucose  chol  \
0     NaN        NaN   NaN         NaN  NaN   NaN   NaN NaN      NaN   NaN   
1     NaN        NaN   NaN         NaN  NaN   NaN   NaN NaN      NaN   NaN   
2     NaN

/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['general_comments', 'heart_comments', 'lung_comments', 'abdomen_comments', 'other_comments']]

  if self.run_code(code, result):
/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['surgical_intervention_detail', 'invasive_intervention_detail', 'imaging_intervention_detail', 'other_intervention_detail']]

  if self.run_code(code, result):
/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object 

day_baseline/CCMI
   score
0      3
1      3
2      3
3      3
4      3
day_baseline/further_stroke_event_part
              arrived  toast  prev_stroke  stroke_num  symp_mot  symp_sen  \
0 2010-08-23 23:40:00      1            0         NaN       NaN       NaN   
1 2010-08-25 04:40:00      1            0         NaN       NaN       NaN   
2 2010-09-01 22:00:00      1            0         NaN       NaN       NaN   
3 2010-10-18 00:54:00      2            0         NaN       NaN       NaN   
4 2010-10-28 04:04:00      1            0         NaN       NaN       NaN   

   symp_vis  symp_lan  symp_other  symp_other_spec stroke_date  ter_lica  \
0       NaN       NaN         NaN              NaN         NaN       NaN   
1       NaN       NaN         NaN              NaN         NaN       NaN   
2       NaN       NaN         NaN              NaN         NaN       NaN   
3       NaN       NaN         NaN              NaN         NaN       NaN   
4       NaN       NaN         NaN             

/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['reason']]

  if self.run_code(code, result):
/Users/alistair/anaconda3/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'pre-stroke_mRS'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['resp_other']]

  if self.run_code(code, result):
/Users/alistair/

   ct_scan        ct_date_time  non_contrast  angioraphy  perfusion  modality  \
0      NaN 2010-08-24 00:20:00           1.0         1.0        1.0       1.0   
1      NaN 2010-08-25 05:00:00           1.0         1.0        1.0       1.0   
2      NaN 2010-09-01 22:17:00           1.0         1.0        1.0       1.0   
3      NaN 2010-10-18 01:30:00           1.0         1.0        1.0       1.0   
4      NaN 2010-10-28 05:10:00           1.0         1.0        1.0       1.0   

   perf_slab  infarct_core_slab1  infarct_core_slab2  infarct_core_total ...   \
0        NaN                 NaN                 NaN                 NaN ...    
1        NaN                 NaN                 NaN                 NaN ...    
2        NaN                 NaN                 NaN                 NaN ...    
3        NaN                 NaN                 NaN                 NaN ...    
4        NaN                 NaN                 NaN                 NaN ...    

   ratio  mis  clinician_i

...

/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['resp_other.3', 'supp_type_other_spec', 'accomodation_other', 'there_other']]

  if self.run_code(code, result):
/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['attempted_not_reason']]

  if self.run_code(code, result):
/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['chg_med_com_1', 'ch

   rec_stroke  rec_stroke_num  rec_stroke_type date_time  toast  img_ct  \
0         0.0             NaN              NaN       NaT    NaN     NaN   
1         0.0             NaN              NaN       NaT    NaN     NaN   
2         NaN             NaN              NaN       NaT    NaN     NaN   
3         0.0             NaN              NaN       NaT    NaN     NaN   
4         0.0             NaN              NaN       NaT    NaN     NaN   

  img_ct_date_time  img_mri  img_mri_date_time  chg_med       ...        \
0              NaT      NaN                NaN      NaN       ...         
1              NaT      NaN                NaN      NaN       ...         
2              NaT      NaN                NaN      NaN       ...         
3              NaT      NaN                NaN      NaN       ...         
4              NaT      NaN                NaN      NaN       ...         

  chg_med_com_3 chg_med_com_3_date chg_med_cea_1 chg_med_cea_1_date  \
0           NaN            

/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['lesion location confirmed by imaging at 3 months (BC report) or if not available medical file', 'first_comments', 'second_comments']]

  if self.run_code(code, result):
/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['not_conducted_reason_other', 'no_mri_sequence_comment']]

  if self.run_code(code, result):



day_365/ARAT
   affected_hand 0=right  dominant 0=left.1  grasp_block10_left  \
0                    NaN                NaN                 NaN   
1                    NaN                NaN                 NaN   
2                    NaN                NaN                 NaN   
3                    NaN                NaN                 NaN   
4                    NaN                NaN                 NaN   

   grasp_block10_right  grasp_block2_left  grasp_block2_right  \
0                  NaN                NaN                 NaN   
1                  NaN                NaN                 NaN   
2                  NaN                NaN                 NaN   
3                  NaN                NaN                 NaN   
4                  NaN                NaN                 NaN   

   grasp_cube5_left  grasp_cube5_right  grasp_cube7_left  grasp_cube7_right  \
0               NaN                NaN               NaN                NaN   
1               NaN               

/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['mri_why_not', 'inj_gad_no_spec']]

  if self.run_code(code, result):
/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['diagnosis', 'outcome_followup', 'valid.1', 'diagnosis.2', 'date_time.5', 'outcome.4']]

  if self.run_code(code, result):


In [71]:
with pd.HDFStore('test.h5') as store:
    store.put('scans',scans)

/Users/alistair/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['p_participant_num', 'site_id', 'initials', 'prefix', 'path']]

  if self.run_code(code, result):


In [66]:
scans.put()

,p_participant_num,site_id,initials,prefix,path
119,01,01,JW,D_PP_P01-01-JW,[build_dataset/correlation_maps/D_PP_P01-01-JW...
123,01,02,AS,D_PP_P02-01-AS,[build_dataset/correlation_maps/D_PP_P02-01-AS...
124,01,12,JW,D_PP_P12-01-JW,[build_dataset/correlation_maps/D_PP_P12-01-JW...
126,10,01,NP,D_PP_P01-10-NP,[build_dataset/correlation_maps/D_PP_P01-10-NP...
127,02,12,AS,D_PP_P12-02-AS,[build_dataset/correlation_maps/D_PP_P12-02-AS...
129,07,01,GS,D_PP_P01-07-GS,[build_dataset/correlation_maps/D_PP_P01-07-GS...
132,02,02,SH,D_PP_P02-02-SH,[build_dataset/correlation_maps/D_PP_P02-02-SH...
133,11,01,GW,D_PP_P01-11-GW,[build_dataset/correlation_maps/D_PP_P01-11-GW...
136,01,15,EM,D_PP_P15-01-EM,[build_dataset/correlation_maps/D_PP_P15-01-EM...
137,02,15,DK,D_PP_P15-02-DK,[build_dataset/correlation_maps/D_PP_P15-02-DK...


In [64]:
with pd.HDFStore('test.h5') as store:
    print(store['/day_0/bloods'].columns)

Index(['Chol', 'Trig', 'HDLC', 'LDLC', 'sFol', 'B12', 'CRP'], dtype='object')


In [22]:
with pd.HDFStore('test.h5') as store:
    p = store['scans']
    print(p['prefix'],p.index)

119    D_PP_P01-01-JW
123    D_PP_P02-01-AS
124    D_PP_P12-01-JW
126    D_PP_P01-10-NP
127    D_PP_P12-02-AS
129    D_PP_P01-07-GS
132    D_PP_P02-02-SH
133    D_PP_P01-11-GW
136    D_PP_P15-01-EM
137    D_PP_P15-02-DK
138    D_PP_P15-03-MB
139    D_PP_P15-04-KT
140    D_PP_P15-05-BC
141    D_PP_P15-07-MP
142    D_PP_P02-03-MH
143    D_PP_P01-14-GC
144    D_PP_P01-15-PM
146    D_PP_P01-16-HK
147    D_PP_P02-06-KF
148    D_PP_P01-17-IB
151    D_PP_P01-19-JS
152    D_PP_P02-07-NS
154    D_PP_P15-10-TS
155    D_PP_P15-12-SR
156    D_PP_P15-13-RF
157    D_PP_P02-08-BL
159    D_PP_P15-14-PS
161    D_PP_P02-10-DE
162    D_PP_P01-21-CA
163    D_PP_P01-22-FM
            ...      
178    D_PP_P12-04-KE
180    D_PP_P01-30-WC
182    D_PP_P01-32-RB
183    D_PP_P15-19-JM
186    D_PP_P15-23-KS
188    D_PP_P15-26-GO
189    D_PP_P15-27-AJ
190    D_PP_P01-33-GL
191    D_PP_P02-20-RA
192    D_PP_P02-21-JT
194    D_PP_P15-28-HK
196    D_PP_P01-37-BM
197    D_PP_P01-38-AB
198    D_PP_P02-22-CW
199    D_P

In [19]:
with open('masks/files.txt') as infile:
    filenames = [line.strip() for line in infile]
    filenames.pop()

In [20]:
filenames_12months = [f for f in filenames if f.endswith('_12months')]

In [21]:
filenames_9months = [f for f in filenames if not f.endswith('_12months')]

In [22]:
len(filenames) == len(filenames_12months) + len(filenames_9months)

True

In [23]:
peter_sheets = sheet_0.join(sheet_1,lsuffix='first',rsuffix='second').join(sheet_2,rsuffix='third')

In [24]:
peter_sheets.drop(labels = ['Deidentifiable IDthird','Deidentifiable ID','Prefixfirst','In list','Prefixsecond','/D_','D_','12 month suffix', 'Out list'],axis = 'columns',inplace=True)

In [25]:
[p for p in peter_sheets.ID]

['D_PP_P01-21-CA',
 'D_PP_P01-21-CA',
 'D_PP_P02-20-RA',
 'D_PP_P02-20-RA',
 'D_PP_P15-03-MB',
 'D_PP_P15-03-MB',
 'D_PP_P01-38-AB',
 'D_PP_P01-38-AB',
 'D_PP_P01-17-IB',
 'D_PP_P01-17-IB',
 'D_PP_P01-24-MB',
 'D_PP_P01-30-WC',
 'D_PP_P01-42-CC',
 'D_PP_P01-42-CC',
 'D_PP_P01-14-GC',
 'D_PP_P15-05-BC',
 'D_PP_P15-05-BC',
 'D_PP_P02-24-AD',
 'D_PP_P02-24-AD',
 'D_PP_P01-39-WD',
 'D_PP_P01-39-WD',
 'D_PP_P15-31-RE',
 'D_PP_P15-31-RE',
 'D_PP_P02-10-DE',
 'D_PP_P02-10-DE',
 'D_PP_P12-04-KE',
 'D_PP_P12-04-KE',
 'D_PP_P02-06-KF',
 'D_PP_P02-06-KF',
 'D_PP_P15-13-RF',
 'D_PP_P15-13-RF',
 'D_PP_P15-29-DG',
 'D_???',
 'D_PP_P01-25-MH',
 'D_PP_P01-25-MH',
 'D_PP_P15-30-AH',
 'D_PP_P15-30-AH',
 'D_PP_P02-02-SH',
 'D_PP_P02-02-SH',
 'D_PP_P02-03-MH',
 'D_PP_P02-03-MH',
 'D_PP_P01-41-JJ',
 'D_PP_P01-41-JJ',
 'D_PP_P15-27-AJ',
 'D_PP_P15-27-AJ',
 'D_PP_P02-15-MK',
 'D_PP_P15-02-DK',
 'D_PP_P15-02-DK',
 'D_PP_P01-43-GK',
 'D_PP_P01-43-GK',
 'D_PP_P15-28-HK',
 'D_PP_P15-28-HK',
 'D_PP_P01-16-HK',
 '

In [26]:
peter_sheets[peter_sheets['ID'] == 'D_???'].values

array([['/GRUNDLE_MARG/2012-10-22_-_GRUNDLE_MARG_002784', 'D_???'],
       ['/McCOLLEY_BARBARA/2012-01-17_-_MCCOLLEY_BARBARA_002922', 'D_???']], dtype=object)

In [27]:
peter_sheets['ID'].shape

(125,)

In [28]:
len(filenames)

125

In [29]:
peter_sheets['ID'][peter_sheets['ID'].str.endswith('MG')]

Series([], Name: ID, dtype: object)

In [35]:
from glob import glob

In [36]:
glob('masks/*')

['masks/files.txt',
 'masks/Untitled.ipynb',
 'masks/output.zip',
 'masks/output',
 'masks/lists.xlsx',
 'masks/id_num.txt',
 'masks/lesion_masks.txt']

In [25]:
mask_list_12months = []
mask_list_9months = []

with open('masks/lesion_masks.txt') as infile:
    for line in infile:
        code_mask = line.strip('/Volumes/neurorehab-rdsi/peter/prepare/rest/output/')
        code_mask_final = code_mask.split('/')[0]
        if code_mask_final.endswith('_12months'):
            mask_list_12months.append(code_mask_final)
        else:
            mask_list_9months.append(code_mask_final)

In [26]:
len(filenames_12months)

60

In [27]:
len(mask_list_12months)

59

In [28]:
len(filenames_9months)

65

In [29]:
len(mask_list_9months)

63

In [40]:
[f for f in filenames_9months if '01-39-W' in f]

['D_PP_P01-39-WD']

In [50]:
process.extractBests('MG',mask_list_12months + mask_list_9months)

[('D_PP_P15-01-EM', 45),
 ('D_PP_P01-22-FM', 45),
 ('D_PP_P01-24-MB', 45),
 ('D_PP_P01-37-BM', 45),
 ('D_PP_P01-07-GS', 45)]

In [55]:
scans

,p_participant_num,site_id,initials,prefix,path
119,01,01,JW,D_PP_P01-01-JW,[build_dataset/correlation_maps/D_PP_P01-01-JW...
123,01,02,AS,D_PP_P02-01-AS,[build_dataset/correlation_maps/D_PP_P02-01-AS...
124,01,12,JW,D_PP_P12-01-JW,[build_dataset/correlation_maps/D_PP_P12-01-JW...
126,10,01,NP,D_PP_P01-10-NP,[build_dataset/correlation_maps/D_PP_P01-10-NP...
127,02,12,AS,D_PP_P12-02-AS,[build_dataset/correlation_maps/D_PP_P12-02-AS...
129,07,01,GS,D_PP_P01-07-GS,[build_dataset/correlation_maps/D_PP_P01-07-GS...
132,02,02,SH,D_PP_P02-02-SH,[build_dataset/correlation_maps/D_PP_P02-02-SH...
133,11,01,GW,D_PP_P01-11-GW,[build_dataset/correlation_maps/D_PP_P01-11-GW...
136,01,15,EM,D_PP_P15-01-EM,[build_dataset/correlation_maps/D_PP_P15-01-EM...
137,02,15,DK,D_PP_P15-02-DK,[build_dataset/correlation_maps/D_PP_P15-02-DK...


In [39]:
'day_0,day_baseline,day_7,day_90,day_365,day_1'.split(',')

['day_0', 'day_baseline', 'day_7', 'day_90', 'day_365', 'day_1']

In [76]:
index_tuples = []
for name,day in zip('day_0,day_baseline,day_7,day_90,day_365,day_1'.split(','),[day_0,day_baseline,day_7,day_90,day_365,day_1]):
    index_tuples.extend([(name,t) for t in day.keys()])
    

In [51]:
name = 'day_0,day_baseline,day_7,day_90,day_365,day_1'.split(',')
day = [day_0,day_baseline,day_7,day_90,day_365,day_1]

df = pd.DataFrame()

index_tuples = []

for d,n in zip(day,name):
    for t in d:
        for c in d[t]:
            index_tuples.append((n,t,c))
            print(n,t,c)
            #print(pd.DataFrame(d[t][c].values,columns=pd.MultiIndex.from_tuples((n,t,c))))

day_0 bloods Chol
day_0 bloods Trig
day_0 bloods HDLC
day_0 bloods LDLC
day_0 bloods sFol
day_0 bloods B12
day_0 bloods CRP
day_0 laboratory_tests date_time
day_0 laboratory_tests hemoglobin
day_0 laboratory_tests white_cell_count
day_0 laboratory_tests platelet_count
day_0 laboratory_tests mcv
day_0 laboratory_tests neutrophil
day_0 laboratory_tests sodium
day_0 laboratory_tests potassium
day_0 laboratory_tests urea
day_0 laboratory_tests creatinine
day_0 laboratory_tests inr
day_0 laboratory_tests egfr
day_0 laboratory_tests aptt
day_0 laboratory_tests pt
day_0 laboratory_tests glucose
day_0 laboratory_tests chol
day_0 laboratory_tests trig
day_0 laboratory_tests hdl
day_0 laboratory_tests ldl
day_baseline physical_examination height
day_baseline physical_examination weight
day_baseline physical_examination temperature
day_baseline physical_examination heart_rate
day_baseline physical_examination respiratory_rate
day_baseline physical_examination systolic_bp
day_baseline physical_exa

day_7 laboratory_tests platelet_count
day_7 laboratory_tests mcv
day_7 laboratory_tests neutrophil
day_7 laboratory_tests sodium
day_7 laboratory_tests potassium
day_7 laboratory_tests urea
day_7 laboratory_tests creatinine
day_7 laboratory_tests inr
day_7 laboratory_tests egfr
day_7 laboratory_tests aptt
day_7 laboratory_tests pt
day_7 laboratory_tests glucose
day_7 laboratory_tests chol
day_7 laboratory_tests trig
day_7 laboratory_tests hdl
day_7 laboratory_tests ldl
day_7 infarct_type is_ich
day_7 infarct_type date_time.1
day_7 infarct_type symptomatic
day_7 infarct_type loc_frontal
day_7 infarct_type loc_temporal
day_7 infarct_type loc_parietal
day_7 infarct_type loc_occipital
day_7 infarct_type loc_subcortical
day_7 infarct_type loc_brainstem
day_7 infarct_type loc_cerebellar
day_7 infarct_type loc_subdural
day_7 infarct_type loc_subarachnoid
day_7 infarct_type treatment
day_7 infarct_type treatment_spec
day_7 concomitant_medications medication_class_id
day_7 concomitant_medicatio

day_90 physical_risk_factors smoke_stop_years
day_90 physical_risk_factors salt
day_90 physical_risk_factors fish
day_90 physical_risk_factors diet
day_90 physical_risk_factors supp
day_90 physical_risk_factors supp_omega3
day_90 physical_risk_factors supp_folate
day_90 physical_risk_factors supp_vitb12
day_90 physical_risk_factors supp_multivit
day_90 physical_risk_factors supp_unknown
day_90 physical_risk_factors supp_other
day_90 physical_risk_factors supp_type_other_spec
day_90 physical_risk_factors breakfast
day_90 physical_risk_factors sleep
day_90 physical_risk_factors mental_stress
day_90 physical_risk_factors work_hr
day_90 physical_risk_factors accomodation
day_90 physical_risk_factors accomodation_other
day_90 physical_risk_factors therapy
day_90 physical_risk_factors there_other
day_90 RAPA resp
day_90 RAPA resp_other
day_90 RAPA resp_date
day_90 RAPA never
day_90 RAPA light
day_90 RAPA light_wk
day_90 RAPA mod_less_30m
day_90 RAPA vig_less_20m
day_90 RAPA mod_more_30m
day_

day_90 concomitant_medications no_meds.24
day_90 concomitant_medications medication_class_id.25
day_90 concomitant_medications duration_units.25
day_90 concomitant_medications duration.25
day_90 concomitant_medications other_spec.24
day_90 concomitant_medications no_meds.25
day_90 concomitant_medications medication_class_id.26
day_90 concomitant_medications duration_units.26
day_90 concomitant_medications duration.26
day_90 concomitant_medications other_spec.25
day_90 concomitant_medications no_meds.26
day_90 concomitant_medications medication_class_id.27
day_90 concomitant_medications duration_units.27
day_90 concomitant_medications duration.27
day_90 concomitant_medications other_spec.26
day_90 concomitant_medications no_meds.27
day_90 MMSE age_assessment
day_90 MMSE pencil
day_90 MMSE watch
day_90 MMSE school_years
day_90 MMSE time_year
day_90 MMSE time_season
day_90 MMSE time_month
day_90 MMSE time_weekday
day_90 MMSE time_date
day_90 MMSE place_state
day_90 MMSE place_country
day_

day_90 ACS leisure_activity38
day_90 ACS leisure_activity38_done
day_90 ACS leisure_activity40
day_90 ACS leisure_activity40_done
day_90 ACS leisure_activity43
day_90 ACS leisure_activity43_done
day_90 ACS leisure_activity45
day_90 ACS leisure_activity45_done
day_90 ACS leisure_activity46
day_90 ACS leisure_activity46_done
day_90 ACS leisure_activity48
day_90 ACS leisure_activity48_done
day_90 ACS leisure_activity50
day_90 ACS leisure_activity50_done
day_90 ACS leisure_activity51
day_90 ACS leisure_activity51_done
day_90 ACS leisure_activity53
day_90 ACS leisure_activity53_done
day_90 ACS leisure_activity54
day_90 ACS leisure_activity54_done
day_90 ACS leisure_activity57
day_90 ACS leisure_activity57_done
day_90 ACS leisure_activity58
day_90 ACS leisure_activity58_done
day_90 ACS leisure_activity60
day_90 ACS leisure_activity60_done
day_90 ACS leisure_activity62
day_90 ACS leisure_activity62_done
day_90 ACS leisure_activity65
day_90 ACS leisure_activity65_done
day_90 ACS leisure_activi

day_365 physical_risk_factors resp.3
day_365 physical_risk_factors resp_other.3
day_365 physical_risk_factors resp_date
day_365 physical_risk_factors waist
day_365 physical_risk_factors smoke_ever
day_365 physical_risk_factors smoke_regular
day_365 physical_risk_factors smoke_regular_years
day_365 physical_risk_factors smoke_now
day_365 physical_risk_factors smoke_now_cigs
day_365 physical_risk_factors smoke_stop
day_365 physical_risk_factors smoke_stop_years
day_365 physical_risk_factors salt
day_365 physical_risk_factors fish
day_365 physical_risk_factors diet
day_365 physical_risk_factors supp
day_365 physical_risk_factors supp_omega3
day_365 physical_risk_factors supp_folate
day_365 physical_risk_factors supp_vitb12
day_365 physical_risk_factors supp_multivit
day_365 physical_risk_factors supp_unknown
day_365 physical_risk_factors supp_other
day_365 physical_risk_factors supp_type_other_spec
day_365 physical_risk_factors breakfast
day_365 physical_risk_factors sleep
day_365 physica

day_365 RAVENs b2_response
day_365 RAVENs b2_correct
day_365 RAVENs b3_response
day_365 RAVENs b3_correct
day_365 RAVENs b4_response
day_365 RAVENs b4_correct
day_365 RAVENs b5_response
day_365 RAVENs b5_correct
day_365 RAVENs b6_response
day_365 RAVENs b6_correct
day_365 RAVENs b7_response
day_365 RAVENs b7_correct
day_365 RAVENs b8_response
day_365 RAVENs b8_correct
day_365 RAVENs b9_response
day_365 RAVENs b9_correct
day_365 RAVENs b10_response
day_365 RAVENs b10_correct
day_365 RAVENs b11_response
day_365 RAVENs b11_correct
day_365 RAVENs b12_response
day_365 RAVENs b12_correct
day_365 RAVENs score
day_365 STROOP dots
day_365 STROOP dot_errors
day_365 STROOP words
day_365 STROOP word_errors
day_365 STROOP colours
day_365 STROOP colour_errors
day_365 STROOP ratio
day_365 shape_cancellation neglect
day_365 shape_cancellation omissions_left
day_365 shape_cancellation omissions_right
day_365 shape_cancellation omissions_total
day_365 TDT lesion location confirmed by imaging at 3 months

day_1 vital_signs observe_time_12
day_1 vital_signs sbp_12
day_1 vital_signs dbp_12
day_1 vital_signs pulse_12
day_1 vital_signs temp_12
day_1 vital_signs observe_time_24
day_1 vital_signs sbp_24
day_1 vital_signs dbp_24
day_1 vital_signs pulse_24
day_1 vital_signs temp_24
day_1 infarct_type is_ich
day_1 infarct_type date_time
day_1 infarct_type symptomatic
day_1 infarct_type loc_frontal
day_1 infarct_type loc_temporal
day_1 infarct_type loc_parietal
day_1 infarct_type loc_occipital
day_1 infarct_type loc_subcortical
day_1 infarct_type loc_brainstem
day_1 infarct_type loc_cerebellar
day_1 infarct_type loc_subdural
day_1 infarct_type loc_subarachnoid
day_1 infarct_type treatment
day_1 infarct_type treatment_spec
day_1 bloods Chol
day_1 bloods Trig
day_1 bloods HDLC
day_1 bloods LDLC
day_1 bloods sFol
day_1 bloods B12
day_1 bloods CRP
day_1 laboratory_tests date_time
day_1 laboratory_tests hemoglobin
day_1 laboratory_tests white_cell_count
day_1 laboratory_tests platelet_count
day_1 labo

In [52]:
index_tuples

[('day_0', 'bloods', 'Chol'),
 ('day_0', 'bloods', 'Trig'),
 ('day_0', 'bloods', 'HDLC'),
 ('day_0', 'bloods', 'LDLC'),
 ('day_0', 'bloods', 'sFol'),
 ('day_0', 'bloods', 'B12'),
 ('day_0', 'bloods', 'CRP'),
 ('day_0', 'laboratory_tests', 'date_time'),
 ('day_0', 'laboratory_tests', 'hemoglobin'),
 ('day_0', 'laboratory_tests', 'white_cell_count'),
 ('day_0', 'laboratory_tests', 'platelet_count'),
 ('day_0', 'laboratory_tests', 'mcv'),
 ('day_0', 'laboratory_tests', 'neutrophil'),
 ('day_0', 'laboratory_tests', 'sodium'),
 ('day_0', 'laboratory_tests', 'potassium'),
 ('day_0', 'laboratory_tests', 'urea'),
 ('day_0', 'laboratory_tests', 'creatinine'),
 ('day_0', 'laboratory_tests', 'inr'),
 ('day_0', 'laboratory_tests', 'egfr'),
 ('day_0', 'laboratory_tests', 'aptt'),
 ('day_0', 'laboratory_tests', 'pt'),
 ('day_0', 'laboratory_tests', 'glucose'),
 ('day_0', 'laboratory_tests', 'chol'),
 ('day_0', 'laboratory_tests', 'trig'),
 ('day_0', 'laboratory_tests', 'hdl'),
 ('day_0', 'laboratory

In [54]:
index = pd.MultiIndex.from_tuples(index_tuples)

In [57]:
df = pd.DataFrame(index=index).T

In [58]:
name = 'day_0,day_baseline,day_7,day_90,day_365,day_1'.split(',')
day = [day_0,day_baseline,day_7,day_90,day_365,day_1]

for d,n in zip(day,name):
    for t in d:
        for c in d[t]:
            df[(n,t,c)] = d[t][c]

In [71]:
df.loc[170,('day_90','SIS',)]

/Users/alistair/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1325: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


resp.2                           0
resp_other.2                   NaN
phy_arm                          4
phy_hand                         3
phy_leg                          3
phy_foot                         3
mem_told                         3
mem_day_bef                      3
mem_to_do                        3
mem_wk_day                       4
mem_concert                      4
mem_think_quick                  4
mem_sol_prob                     4
feel_sad                         5
feel_nobody                      5
feel_burden                      5
feel_nothing                     5
feel_blame                       5
feel_enjoy                       1
feel_nervous                     5
feel_life_with                   1
feel_smile                       1
com_say_name                     5
com_under                        5
com_reply                        5
com_correct                      5
com_part                         5
com_ph                           5
com_ph_num          

In [73]:
df.to_hdf('big_multi_index.h5','behavioural')

/Users/alistair/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block3_values] [items->[('day_baseline', 'physical_examination', 'general_comments'), ('day_baseline', 'physical_examination', 'heart_comments'), ('day_baseline', 'physical_examination', 'lung_comments'), ('day_baseline', 'physical_examination', 'abdomen_comments'), ('day_baseline', 'physical_examination', 'other_comments'), ('day_baseline', 'recent_medical_history', 'surgical_intervention_detail'), ('day_baseline', 'recent_medical_history', 'invasive_intervention_detail'), ('day_baseline', 'recent_medical_history', 'imaging_intervention_detail'), ('day_baseline', 'recent_medical_history', 'other_intervention_detail'), ('day_baseline', 'past_medical_history', 'Med1_condition_name'), ('day_baseline', 'past_medical_history', 'Med2_condition_name'), 

In [74]:
df_2 = pd.read_hdf('big_multi_index.h5')

In [91]:
with pd.HDFStore('big_multi_index.h5') as h:
    print(h['behavioural'])

     day_0                                                                    \
    bloods                                       laboratory_tests              
      Chol Trig HDLC LDLC  sFol    B12    CRP           date_time hemoglobin   
0      NaN  NaN  NaN  NaN   NaN    NaN    NaN                 NaT        NaN   
1      NaN  NaN  NaN  NaN   NaN    NaN    NaN                 NaT        NaN   
2      NaN  NaN  NaN  NaN   NaN    NaN    NaN                 NaT        NaN   
3      NaN  NaN  NaN  NaN   NaN    NaN    NaN                 NaT        NaN   
4      NaN  NaN  NaN  NaN   NaN    NaN    NaN                 NaT        NaN   
5      NaN  NaN  NaN  NaN   NaN    NaN    NaN                 NaT        NaN   
6      NaN  NaN  NaN  NaN   NaN    NaN    NaN                 NaT        NaN   
7      NaN  NaN  NaN  NaN   NaN    NaN    NaN                 NaT        NaN   
8      NaN  NaN  NaN  NaN   NaN    NaN    NaN                 NaT        NaN   
9      NaN  NaN  NaN  NaN   NaN    NaN  

In [95]:
df_2.columns.levshape

(6, 39, 1447)

In [89]:
index.levels[1]

Index(['ACS', 'ARAT', 'BI', 'CCMI', 'MADRS', 'MMSE', 'MRI', 'MoCA', 'NIHSS',
       'NIHSS_multiple', 'RAPA', 'RAVENs', 'SIS', 'STROOP', 'TDT', 'TUGT',
       'TUGT_average', 'WSAS', 'bloods', 'concomitant_medications',
       'concomitant_medications_plus', 'diet_q', 'digit_span',
       'further_stroke_event_part', 'history_of_depression', 'infarct_type',
       'laboratory_tests', 'lesion_location_confirmed', 'mRS',
       'past_medical_history', 'physical_examination', 'physical_risk_factors',
       'pre-stroke_mRS', 'radiological_scans', 'recent_medical_history',
       'serious_adverse_event', 'shape_cancellation', 'trail_making_test_b',
       'vital_signs'],
      dtype='object')

In [83]:
df_2.

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            209, 210, 211, 212, 213, 214, 215, 216, 217, 218],
           dtype='int64', length=219)

In [88]:
name = 'day_0,day_baseline,day_7,day_90,day_365,day_1'.split(',')
day = [day_0,day_baseline,day_7,day_90,day_365,day_1]

for data,name in zip(day,name):
    print(name)
    print([(test,[c for c in cols.values]) for test,cols in  zip(data.keys(),[data[k].columns for k in data])])

day_0
[('bloods', ['Chol', 'Trig', 'HDLC', 'LDLC', 'sFol', 'B12', 'CRP']), ('laboratory_tests', ['date_time', 'hemoglobin', 'white_cell_count', 'platelet_count', 'mcv', 'neutrophil', 'sodium', 'potassium', 'urea', 'creatinine', 'inr', 'egfr', 'aptt', 'pt', 'glucose', 'chol', 'trig', 'hdl', 'ldl'])]
day_baseline
[('physical_examination', ['height', 'weight', 'temperature', 'heart_rate', 'respiratory_rate', 'systolic_bp', 'diastolic_bp', 'general', 'general_comments', 'heart', 'heart_comments', 'lungs', 'lung_comments', 'abdomen', 'abdomen_comments', 'other', 'other_comments', 'perform_ecg', 'perform_ecg_result']), ('NIHSS_multiple', ['nihss_three_hr', 'nihss_three_hr_score', 'nihss_three_hr_datetime', 'nihss_seven_hr', 'nihss_seven_hr_score', 'nihss_seven_hr_datetime', 'nihss_eighteen_hr', 'nihss_eighteen_hr_score', 'nihss_eighteen_hr_datetime', 'nihss_other_hr', 'nihss_other_hr_score', 'nihss_other_hr_datetime']), ('recent_medical_history', ['bp_three_hr', 'bp_three_hr_systolic', 'bp_t

In [79]:
index_m = pd.MultiIndex.from_tuples(index_tuples,names=['time','test'])

In [62]:
h_df.index.levels

FrozenList([['day_0', 'day_1', 'day_365', 'day_7', 'day_90', 'day_baseline'], ['ACS', 'ARAT', 'BI', 'CCMI', 'MADRS', 'MMSE', 'MRI', 'MoCA', 'NIHSS', 'NIHSS_multiple', 'RAPA', 'RAVENs', 'SIS', 'STROOP', 'TDT', 'TUGT', 'TUGT_average', 'WSAS', 'bloods', 'concomitant_medications', 'concomitant_medications_plus', 'diet_q', 'digit_span', 'further_stroke_event_part', 'history_of_depression', 'infarct_type', 'laboratory_tests', 'lesion_location_confirmed', 'mRS', 'past_medical_history', 'physical_examination', 'physical_risk_factors', 'pre-stroke_mRS', 'radiological_scans', 'recent_medical_history', 'serious_adverse_event', 'shape_cancellation', 'trail_making_test_b', 'vital_signs']])

In [80]:
h_df = pd.DataFrame(index=index_m)

In [81]:
h_df.loc['day_0'].T

Empty DataFrame
Columns: [(day_0, bloods), (day_0, laboratory_tests), (day_baseline, physical_examination), (day_baseline, NIHSS_multiple), (day_baseline, recent_medical_history), (day_baseline, past_medical_history), (day_baseline, CCMI), (day_baseline, further_stroke_event_part), (day_baseline, NIHSS), (day_baseline, pre-stroke_mRS), (day_baseline, bloods), (day_baseline, laboratory_tests), (day_baseline, radiological_scans), (day_baseline, concomitant_medications), (day_7, bloods), (day_7, NIHSS), (day_7, history_of_depression), (day_7, MoCA), (day_7, MADRS), (day_7, physical_risk_factors), (day_7, diet_q), (day_7, RAPA), (day_7, laboratory_tests), (day_7, infarct_type), (day_7, concomitant_medications), (day_7, concomitant_medications_plus), (day_90, bloods), (day_90, mRS), (day_90, NIHSS), (day_90, BI), (day_90, SIS), (day_90, physical_examination), (day_90, physical_risk_factors), (day_90, RAPA), (day_90, WSAS), (day_90, diet_q), (day_90, laboratory_tests), (day_90, further_stroke_event_part), (day_90, history_of_depression), (day_90, MoCA), (day_90, MADRS), (day_90, concomitant_medications), (day_90, MMSE), (day_90, TUGT), (day_90, digit_span), (day_90, RAVENs), (day_90, STROOP), (day_90, shape_cancellation), (day_90, TDT), (day_90, ARAT), (day_90, TUGT_average), (day_90, ACS), (day_90, MRI), (day_90, lesion_location_confirmed), (day_365, bloods), (day_365, mRS), (day_365, NIHSS), (day_365, BI), (day_365, SIS), (day_365, physical_examination), (day_365, physical_risk_factors), (day_365, RAPA), (day_365, WSAS), (day_365, diet_q), (day_365, laboratory_tests), (day_365, further_stroke_event_part), (day_365, history_of_depression), (day_365, MoCA), (day_365, MADRS), (day_365, concomitant_medications), (day_365, MMSE), (day_365, trail_making_test_b), (day_365, digit_span), (day_365, RAVENs), (day_365, STROOP), (day_365, shape_cancellation), (day_365, TDT), (day_365, ARAT), (day_365, TUGT), (day_365, ACS), (day_365, MRI), (day_1, NIHSS), (day_1, MRI), (day_1, vital_signs), (day_1, infarct_type), (day_1, bloods), (day_1, laboratory_tests), (day_1, serious_adverse_event)]
Index: []

[0 rows x 88 columns]

In [70]:
h_df.loc[('day_0','bloods'),[c for c in day_0['bloods'].columns]] = day_0['bloods']

KeyError: "['Chol' 'Trig' 'HDLC' 'LDLC' 'sFol' 'B12' 'CRP'] not in index"